## Mercedes-Benz 데이터셋

- 많은 공정 변수가 포함되어 있는 데이터셋을 통해서, 차원의 저주 문제를 해결해보고 여러가지 전처리 기법들을 적용해봅니다.

## 1. 라이브러리, 데이터 불러오기

In [ ]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# flag variables
remove_outlier = True
encoding = 'OH' # 'OD', 'OH'
feature_reducing = 2   # 'PCA':0 / 'correlation':1 / 'importance':2 (optional)

In [ ]:
# 데이터를 불러옵니다.
train = pd.read_csv('../input/mercedes-benz-greener-manufacturing/train.csv.zip')
test = pd.read_csv('../input/mercedes-benz-greener-manufacturing/test.csv.zip')
print(train.shape, test.shape)

## 2. EDA

#### 찾은 특징들


1. 결측치 : 없음


2. dtype이 object인 column : X0 ~ X8까지 8개. (categorical feature)

> -> 어떻게 처리할지 고민해야함. (Ordinal Encoding VS One-Hot Encoding)

> -> categorical feature들은 종류 정보들이 알파벳으로 되어있으며(anomynized) 이 정보들 대비 target값의 차이가 있는지 확인.
(특별하게 관련 없음)

> -> binary feature들중에서 0만 가지고 있는 column들이 있음.

> -> 정보가 충분하지 않다고 판단(target value와의 관련성 0) 삭제.


3. target distribution
-> train data에 180을 넘는 데이터가 하나 있음. 이 데이터를 outlier라고 생각하고 제거.

In [ ]:
# 1. 결측치 체크 --> train, test 데이터 모두 결측치가 존재하지 않음.

train.loc[:, train.isnull().any()]   # 결측치를 포함하고 있는 column의 모든 row를 출력.
train.columns[train.isnull().any()]  # 결측치를 포함하고 있는 모든 column을 출력.
train[train.isnull().any(axis=1)]    # 결측치를 포함하고 있는 모든 row를 출력.

test[test.isnull().any(axis=1)]

In [ ]:
# 2. dtype이 object인 column 확인 (categorical feature 찾기)
#cat_features = ["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]
cat_features = train.columns[train.dtypes == 'object']
cat_features

In [ ]:
# categorical feature들 확인
for col in cat_features:
    plt.figure(figsize=(12, 6))
    sns.countplot(data=train, x=col) # X0 ~ X8
    plt.show()

In [ ]:
# 2-2. binary features
## TO-DO : binary feature들에 대해서, 모두 0 또는 모두 1을 가지고 있는 column들을 찾아서 제거!

# 1) 전체 column에서 ID, y, cat_feature를 제외하는 방법.
binary_features = train.columns.drop(['ID', 'y'] + list(cat_features))

# 2) 전체 column에 dtypes이 int인 column을 뽑는 방법.
binary_features = train.columns[train.dtypes == 'int64'].drop("ID")

# 모두 0이거나 모두 1인 column들.
drop_cols = binary_features[(train[binary_features].sum() == 0) | (train[binary_features].sum() == len(train))]
drop_cols

In [ ]:
# 3. target distribution
plt.figure(figsize=(10, 6))
sns.histplot(data=train, x="y", bins=100)
plt.show()

In [ ]:
if remove_outlier:
    y_limit = 175
    train = train.drop(index=train[train.y > y_limit].index)

### 3. 전처리

#### 결측치 처리

- 비어있는 값들이 있는 column은 어떻게 처리할까요?


> 결측치 없음!

In [ ]:
# 결측치가 있는 column
pass

In [ ]:
# binary_feature들 중에서 필요없는 feature들 제거
train = train.drop(columns=drop_cols)
test  = test.drop(columns=drop_cols)
print(train.shape, test.shape)

#### feature 구분

- X0 ~ X8 : categorical feature

- other features : binary feature(0 / 1)

In [ ]:
# categorical feature encoding
# 1) Ordinal Encoding

# type_map = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6}
# train.X3 = train.X3.map(type_map)  # custom mapping

# 전처리를 위해서 train, test 데이터를 합친 데이터를 하나 생성.
total = pd.concat([train, test])

if encoding == 'OD':
    for col in cat_features:
        total[col] = pd.factorize(total[col])[0]
    

# 2) One-hot Encoding
elif encoding == "OH":
    total = pd.get_dummies(data=total, columns=cat_features)

    
train = total[:len(train)] # 4209 
test = total[len(train):].drop(columns="y") # 4209
print(train.shape, test.shape)

### feature engineering

1. PCA(Principal Component Analysis)


2. Correlation


3. VIF


4. (OPTIONAL) feature importance

In [ ]:
# 1. PCA를 이용하여 차원 감소 시키기
if feature_reducing == 0:
    from sklearn.decomposition import PCA

    #pca = PCA(n_components=15, random_state=42)   # 15차원으로 변환해주세요.
    pca = PCA(n_components=0.95, random_state=42) # 원본 데이터의 정보를 95% 보존하는 차원으로 변환해주세요.
    pca_X = pca.fit_transform(train.drop(columns=["ID", 'y']))

    pca_columns = [f"PC{i}" for i in range(1, pca_X.shape[1]+1)]
    X = pd.DataFrame(data=pca_X, columns=pca_columns)
    display(X)

In [ ]:
# 2. correlation을 이용하여 서로 비슷한 정보를 가지는 피처 제거하기
if feature_reducing == 1:
    threshold = 0.9
    except_cols = []
    remove_cols = []

    corr = train.drop(columns=["ID", 'y']).corr()

    for col in corr: # X0 -> X385

        if col in except_cols:
            continue

        except_cols.append(col)
        row = np.abs(corr.loc[col])   # 절대값을 취해서 양/음수를 없앰. (크기만 비교하겠다)
        condition1 = (row >= threshold)
        condition2 = ~corr.columns.isin(except_cols)  # 전체 column중에 except_cols에 없는 column들.

        temp = row[condition1 & condition2].index  # 위의 두 조건을 모두 만족하는 column들 (v)
        except_cols = except_cols + list(temp)
        remove_cols = remove_cols + list(temp)

    print("Number of columns to be removed : ", len(remove_cols))
    
    X = train.drop(columns=["ID", 'y'] + remove_cols)

In [ ]:
# # 3. VIF 구현하기
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# # VIF_i = 1 / (1 - R^2_i)

# X = train.drop(columns=["ID", 'y'])

# vifs = [variance_inflation_factor(X, idx) for idx in range(len(X.columns))]
# vif_df = pd.DataFrame({"feature_name":X.columns, "VIF":vifs})
# vif_df.sort_values(by="VIF", ascending=False)

# # vifs = [variance_inflation_factor(X, idx) for idx in range(8)]
# # vif_df = pd.DataFrame({"feature_name":X.columns[:8], "VIF":vifs})
# # vif_df.sort_values(by="VIF", ascending=False)

In [ ]:
# # VIF가 threshold를 넘는 경우에 해당 column을 제거.
# threshold = 10
# temp_cols = vif_df.loc[vif_df.VIF >= threshold, "feature_name"].values

# X = X.drop(columns=temp_cols)

In [ ]:
# 4. (OPTIONAL) feature importance를 기준으로 중요한 feature 뽑기
if feature_reducing == 2:
    X = train.drop(columns=["ID", 'y'])
    y = train.y

    from sklearn.ensemble import RandomForestRegressor

    reg = RandomForestRegressor(n_estimators=100,
                               max_depth=15,
                               max_features=0.7)
    reg.fit(X, y)

    importances = reg.feature_importances_
    feature_names = X.columns
    indices = np.argsort(importances)[::-1][:15] # feature importance가 높은 Feature들의 index

    plt.figure(figsize=(10, 6))
    sns.barplot(x=feature_names[indices], y=importances[indices])
    plt.show()
    
    X = X[feature_names[indices]] # feature importance가 높은 15개를 선정.

### 전처리가 완료된 X, y를 세팅합니다!

In [ ]:
X_train = X
y_train = train.y

In [ ]:
X_train